# CLI

In [ ]:
#| default_exp cli

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
# |hide
import platform

import torch
import timm
import wandb
from fastcore.script import call_parse

from gpu_sprint.benchmark import *

In [ ]:
# |export
@call_parse
def cli(wnb: str ="disabled",       # W&B mode. Accepted values: online, offline, disabled.
        wnb_run: str =None,    # W&B run name (auto-generate if None)
        wnb_project: str =None,
        wnb_entity: str =None,
        model: str ="resnet50",     # TIMM Model name
        bs: int =32,                #Batch size
        size: int =224,             # (fake) image size
        fp16: bool =False,
        n_batches = 0,              # Run for N batches. Mututally exclusive with `n_seconds`
        n_seconds = 0,              # Run for N seconds. Mutually exclusive with `n_batches`
    ):

    if not n_batches and not n_seconds:
        print("Either `n_batches` or `n_seconds` must be non-zero")
        exit(1)
    if n_batches and n_seconds:
        print(f"`n_batches` and `n_seconds` are mutually explusive. {n_batches=}, {n_seconds=}")
        exit(1)

    assert wnb in ["online", "offline", "disabled"]
    
    stats = {
        "device_name" : torch.cuda.get_device_name(),
        "device_capability": str(torch.cuda.get_device_capability()),
        "pytorch": torch.version.__version__,
        "cuda": torch.version.cuda,
        "platform": platform.platform(),
        "model": model,
        "fp16": fp16,
        "bs": bs,
        "n_seconds": n_seconds if n_seconds else None,
        "n_batches": n_batches if n_batches else None,
    }

    print("\n".join([ f"{k}: {v}" for k, v in stats.items() ]))

    run = wandb.init(mode=wnb, project=wnb_project, entity=wnb_entity, name=wnb_run)

    with run:
        wandb.log(stats, step=0)

        model = timm.create_model(model, pretrained=False)
        duration, n_items = benchmark(model, bs=bs, size=size, fp16=fp16, n_batches=n_batches, n_seconds=n_seconds)
        summary = {
            "duration": duration,
            "n_items": n_items,
            "throughput": n_items / duration,
        }

        print("\n".join([ f"{k}: {v}" for k, v in summary.items() ]))

        wandb.log(summary, step=0)



In [ ]:
# |eval: false
# |hide
cli(wnb = "disabled",
    wnb_run="hbfs",
    wnb_project="hbfs",
    model="resnet50",
    bs=32,
    size=224,
    fp16=False,
    n_batches = 0,
    n_seconds = 20)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()